# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producteur:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
ventes:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [5]:
%%sql DROP TABLE IF EXISTS producer;
DROP TABLE IF EXISTS wine;
DROP TABLE IF EXISTS production;
DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS sales;

-- Create producer table
CREATE TABLE producer AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV,CRU,DEGRE,MILL,QTE
FROM MASTER1
WHERE NV IS NOT NULL;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV,NP,QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;

-- Create customer table
CREATE TABLE customer AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE,REGION
FROM MASTER2
WHERE NB IS NOT NULL;

-- Create sales table
CREATE TABLE sales AS
SELECT DISTINCT NV,NB,DATES,LIEU,QTE
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [6]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT NB,NOM,PRENOM,TYPE
FROM customer
WHERE TYPE = 'gros' OR TYPE = 'moyen';



 * sqlite:///wine.db
Done.
Done.


[]

In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
3,Aron,Raymond,gros
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
8,Aragon,Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [9]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT DISTINCT *
FROM customer
WHERE NB NOT IN (
    SELECT NB
    FROM sales);

 * sqlite:///wine.db
Done.
Done.


[]

In [10]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,REGION
11,Breton,Andre,petit,None
13,Barthes,Roland,moyen,None
16,Balzac,Honore de,moyen,None
18,Celine,Louis Ferdinand,gros,None
20,Chateaubriand,Francois-Rene de,moyen,None
21,Corbiere,Tristan,petit,None
23,Corneille,Pierre,petit,None
25,Char,Rene,petit,None
27,Dumas,Alexandre,gros,None
29,Fournier,Alain,petit,None


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [11]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT DISTINCT *
FROM customer
WHERE NB IN (
    SELECT NB
    FROM sales);

 * sqlite:///wine.db
Done.
Done.


[]

In [12]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,REGION
1,Aristote,None,petit,ÎLE-DE-FRANCE
2,Artaud,Antonin,moyen,NOUVELLE-AQUITAINE
2,Artaud,Antonin,moyen,BOURGOGNE-FRANCHE-COMTÉ
3,Aron,Raymond,gros,GRAND EST
3,Aron,Raymond,gros,NOUVELLE-AQUITAINE
3,Aron,Raymond,gros,ÎLE-DE-FRANCE
4,Apollinaire,Guillaume,moyen,ÎLE-DE-FRANCE
5,Audiberti,Jacques,petit,AUVERGNE-RHÔNE-ALPES
5,Audiberti,Jacques,petit,ÎLE-DE-FRANCE
5,Audiberti,Jacques,petit,NORMANDIE


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [13]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT s.LIEU AS LIEU, w.CRU AS CRU, SUM(s.QTE) AS QTE_BUE
FROM wine w
INNER JOIN sales s
ON w.NV = s.NV
INNER JOIN customer c
ON c.NB = s.NB
WHERE s.DATES like '1983%'
GROUP BY LIEU, CRU

 * sqlite:///wine.db
Done.
Done.


[]

In [14]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,9
LILLE,Pommard,5
LYON,Beaujolais Villages,30
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,2
PARIS,Pouilly Vinzelles,6
RENNES,Mercurey,2
ROCQUENCOURT,Beaujolais Villages,1000
ROCQUENCOURT,Saint Amour,240


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [15]:
%%sql
DROP TABLE IF EXISTS RBB;

CREATE TABLE RBB AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM bons_buveurs;

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [17]:
%%sql

SELECT c.NB AS NB, c.NOM AS NOM, c.PRENOM AS PRENOM, SUM(s.QTE) AS QTE_BOUGHT
FROM customer c
INNER JOIN sales s
ON c.NB = s.NB
GROUP BY c.NB, c.NOM, c.PRENOM
HAVING QTE_BOUGHT >= 100

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,QTE_BOUGHT
2,Artaud,Antonin,1166
3,Aron,Raymond,174
5,Audiberti,Jacques,339
8,Aragon,Louis,264
9,Ajar,Emile,280
28,Dostoievsky,Fiodor,100
44,Gide,Andre,171


Update instances

In [18]:
%%sql

UPDATE customer
SET TYPE = 'gros'
WHERE NB IN (
    SELECT NB 
    FROM (
        SELECT c.NB AS NB, SUM(s.QTE) AS QTE_BOUGHT
        FROM customer c
        INNER JOIN sales s
        ON c.NB = s.NB
        GROUP BY c.NB, c.NOM, c.PRENOM
        HAVING QTE_BOUGHT >= 100
    )
)

 * sqlite:///wine.db
14 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [19]:
%%sql

SELECT COUNT(*) FROM bons_buveurs
WHERE TYPE = 'gros';


 * sqlite:///wine.db
Done.


COUNT(*)
45


In [22]:


%%sql  

SELECT COUNT(*) FROM RBB
WHERE TYPE = 'gros';



 * sqlite:///wine.db
Done.


COUNT(*)
34


In [21]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


In [20]:
%%sql  

SELECT COUNT(*) FROM RBB
WHERE TYPE = 'gros';


 * sqlite:///wine.db
Done.


COUNT(*)
34


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [23]:
%%sql
DROP TABLE IF EXISTS RBA;

CREATE TABLE RBA AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM buveurs_asec;

 * sqlite:///wine.db
Done.
Done.


[]

In [24]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [25]:
%%sql

INSERT INTO RBA (NB, NOM, PRENOM, TYPE)
VALUES (101, 'Reynal', 'Stephane', 'moyen')

 * sqlite:///wine.db
1 rows affected.


[]

In [26]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [27]:
%%sql

SELECT COUNT(*) FROM RBA;

 * sqlite:///wine.db
Done.


COUNT(*)
58


In [28]:
%%sql

SELECT COUNT(*) FROM buveurs_asec;

 * sqlite:///wine.db
Done.


COUNT(*)
57


___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.